In [1]:
import csv
from collections import defaultdict

connections = defaultdict(dict)

with open('simulation/simulation.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        metric = row[1]
        
        connections[metric]["type"] = "metric"
        connections[metric]['inputs'] = []  # Make it a list for multiple inputs
        connections[metric]['outputs'] = []
        connections[metric]['min'] = float(row[4]) if row[4] else None
        connections[metric]['max'] = float(row[5]) if row[5] else None
        connections[metric]['default'] = float(row[3]) if row[6] else None

        hash_indices = [i for i, x in enumerate(row) if x == '#']

        if len(hash_indices) > 1 and row[hash_indices[1] + 1]:
            input_formulas = row[hash_indices[1] + 1:hash_indices[2]] if len(hash_indices) > 2 else row[hash_indices[1] + 1:]
            for input_formula in input_formulas:
                if ',' in input_formula:
                    input_metric, input_equation = input_formula.split(',', 1)
                    connections[metric]['inputs'].append({'name': input_metric, 'equation': input_equation.split(',')[0]})  # Append to list

        if len(hash_indices) > 2:
            for output_formula in row[hash_indices[2] + 1:]:
                if output_formula and ',' in output_formula:
                    output_metric, output_equation = output_formula.split(',', 1)
                    connections[metric]['outputs'].append({'name': output_metric, 'equation': output_equation.split(',')[0]})

metrics = connections
metrics["WorkerProductivity"]

{'type': 'metric',
 'inputs': [],
 'outputs': [{'name': 'GDP', 'equation': '0.35*(x^0.76)'},
  {'name': 'InternationalTrade', 'equation': '-0.2+(0.4*x)'},
  {'name': 'Unemployment', 'equation': '-0.08+(0.16*x)'}],
 'min': 0.0,
 'max': 1.0,
 'default': 0.45}

In [2]:
from collections import defaultdict
import csv

# Initialize a dictionary to hold the policy effects
policy_effects = defaultdict(list)

# Read the CSV file and map the effects
with open('simulation/policies.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header
    for row in reader:
        policy_name = row[1]  # The policy name is in the second column
        if row[10].startswith('_prereq_sovereign'):
            continue

        # Find the index of '#Effects' to locate policy effects
        try:
            effects_start_idx = row.index('#Effects') + 1
        except ValueError:
            continue

        # Parse the output effects and add to the dictionary
        for effect in row[effects_start_idx:]:
            if effect and ',' in effect:
                effect_metric, effect_equation = effect.split(',', 1)
                policy_effects[policy_name].append({'name': effect_metric, 'equation': effect_equation})

policy_effects
len(policy_effects)

266

In [3]:
# Initialize a dictionary to hold all relationships
all_relationships = defaultdict(lambda: {'inputs': [], 'outputs': []})

# Populate relationships from metrics
for metric, details in connections.items():
    all_relationships[metric]["type"] = "metric"
    all_relationships[metric]["min"] = details['min']
    all_relationships[metric]["max"] = details['max']
    all_relationships[metric]["default"] = details['default']

    # Handle input
    input_details = details.get('inputs', [])
    if input_details:
        for input_ in input_details:
            input_metric = input_['name']
            all_relationships[metric]['inputs'].append({
                'name': input_metric,
                'equation': input_['equation']
            })

    # Handle outputs
    output_details = details.get('outputs', [])
    if output_details:
        for output in output_details:
            output_metric = output['name']
            all_relationships[metric]['outputs'].append({
                'name': output_metric,
                'equation': output['equation'].split(',')[0]
            })

# Populate relationships from policy effects
for policy, effects in policy_effects.items():
    if effects:
        for effect in effects:
            effect_metric = effect['name']
            all_relationships[policy]["type"] = "policy"
            all_relationships[policy]['outputs'].append({
                'name': effect_metric,
                'equation': effect['equation'].split(',')[0]
            })

# Now, all_relationships contains all the interconnected relationships
all_relationships["WorkerProductivity"]

{'inputs': [],
 'outputs': [{'name': 'GDP', 'equation': '0.35*(x^0.76)'},
  {'name': 'InternationalTrade', 'equation': '-0.2+(0.4*x)'},
  {'name': 'Unemployment', 'equation': '-0.08+(0.16*x)'}],
 'type': 'metric',
 'min': 0.0,
 'max': 1.0,
 'default': 0.45}

In [4]:
import csv

situations = {}

# Open the situations.csv file and read its content
with open('simulation/situations.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row

    for row in reader:
        if all(x == '' for x in row):
            continue  # Skip empty rows

        name = row[1]
        if name in ["Monarchy", "NorthKorea", "NorthWestPassage", "Malaria", "MineralWealth", "OilWealth"]:
            continue
        
        positive = True if row[5] == '1' else False
        start_trigger_value = float(row[6])
        stop_trigger_value = float(row[7])

        # Initialize empty dictionaries for inputs and outputs
        situations[name] = {
            'positive': positive, 
            'start_trigger_value': start_trigger_value, 
            'stop_trigger_value': stop_trigger_value, 
            'inputs': [], 
            'outputs': []
        }

        # Find indices of '#' to separate inputs and outputs
        hash_indices = [i for i, x in enumerate(row) if x == '#']

        # Handle inputs if they exist
        if len(hash_indices) > 0:
            for input_formula in row[hash_indices[0] + 1:hash_indices[1]]:
                if input_formula and ',' in input_formula:
                    input_metric, input_equation = input_formula.split(',', 1)
                    situations[name]['inputs'].append({'type': 'metric', 'name': input_metric, 'equation': input_equation.split(',')[0]})

        # Handle outputs if they exist
        if len(hash_indices) > 1:
            for output_formula in row[hash_indices[1] + 1:]:
                if output_formula and ',' in output_formula:
                    output_metric, output_equation = output_formula.split(',', 1)
                    situations[name]['outputs'].append({'type': 'metric', 'name':output_metric, 'equation': output_equation.split(',')[0]})

# Assuming all_relationships, metrics, policy_effects, and situations are already defined
for situation_name, situation_data in situations.items():
    all_relationships[situation_name] = {
        'type': 'situation',
        'inputs': situation_data['inputs'],
        'outputs': situation_data['outputs'],
        'start_trigger': situation_data['start_trigger_value'],
        'stop_trigger': situation_data['stop_trigger_value'],
    }

In [5]:

from scipy.optimize import minimize
import re
from pyswarm import pso


# Utility function to evaluate an equation safely
def evaluate_equation(equation, x_value, policy_values, metrics_values, metric_name, all_relationships, in_progress):
    equation = equation.replace('^', '**')
    equation = equation.replace('x', str(x_value))
    
    for match in re.findall(r'\b([a-zA-Z_]+)\b', equation):
        if match in policy_values:
            equation = equation.replace(match, str(policy_values[match]))
        elif match in metrics_values:
            equation = equation.replace(match, str(metrics_values[match]))
        else:
            computed_value = compute_metric(match, all_relationships, policy_values, metrics_values, in_progress=in_progress)
            metrics_values[match] = computed_value
            equation = equation.replace(match, str(computed_value))

    try:
        res = 0
        if isinstance(res, complex):
            res = 0  # Replace with your real-world behavior
        res = round(eval(equation), 2)
        return res

    except Exception as e:
        raise ValueError(f"Error in evaluating equation: {e}")


# Recursive function to compute the value of a target metric
MAX_DEPTH = 100  # Maximum recursion depth to avoid infinite loops
debug_info = []

def compute_metric(target, all_relationships, policy_values, metrics_values, depth=0, in_progress=None, reset_metrics=None):   
    global debug_info     
    if in_progress is None:
        in_progress = set()
        
    if target in ["_globaleconomy_", "_default_", "_year"]:
        return 0.5
    
    if reset_metrics is not None and target in reset_metrics:
        metrics_values.pop(target, None)
        
    if target in in_progress or depth > MAX_DEPTH:
        if target in metrics_values:
            return metrics_values[target]
        if target in policy_values:
            return policy_values[target]
        
        return 0

    in_progress.add(target)
    value = all_relationships.get(target, {}).get('default', 0)

    if target in metrics_values:
        return metrics_values[target]
    
    if target in policy_values:
        return policy_values[target]

    inputs = all_relationships.get(target, {}).get('inputs', [])
    for inp in inputs:
        metric_name = inp['name']
        equation = inp['equation']
        input_value = compute_metric(metric_name, all_relationships, policy_values, metrics_values, depth + 1, in_progress, reset_metrics)
        if not metric_name in list(situations.keys()) or (metric_name in list(situations.keys()) and input_value >= all_relationships[metric_name]['start_trigger']):
            eq_value = evaluate_equation(equation, input_value, policy_values, metrics_values, metric_name, all_relationships, in_progress)
            value += round(eq_value, 2)
            metrics_values[target] = value
        
            if target in DEBUG_VALUE:
                debug_info.append((metric_name, input_value, eq_value, target))


    for key, rel in all_relationships.items():
        outputs = rel.get('outputs', [])
        for output in outputs:
            if output['name'] == target:
                metric_name = key
                equation = output['equation']
                input_value = compute_metric(metric_name, all_relationships, policy_values, metrics_values, depth + 1, in_progress, reset_metrics)
                
                if not metric_name in list(situations.keys()) or (metric_name in list(situations.keys()) and input_value >= all_relationships[metric_name]['start_trigger']):
                    eq_value = evaluate_equation(equation, input_value, policy_values, metrics_values, metric_name, all_relationships, in_progress)
                    value += round(eq_value, 2)
                    metrics_values[target] = value
                
                    if target in DEBUG_VALUE:
                        debug_info.append((metric_name, input_value, eq_value, target))

    min_val = all_relationships.get(target, {}).get('min', 0)
    max_val = all_relationships.get(target, {}).get('max', 1)

    res = round(value, 2)
    if target in DEBUG_VALUE:
        debug_info.sort(key=lambda x: x[2], reverse=True)
        for metric_name, input_value, eq_value, target in debug_info:
            print(f"Metric: {metric_name} (for {target}), Input: {input_value}, Impact: {eq_value}")
        print(f"Final Calculation for {target}: {res}")
        debug_info = []
        
    if res < min_val:
        res = min_val
    elif res > max_val:
        res = max_val
    metrics_values[target] = res
    if reset_metrics is not None and target in reset_metrics:
        reset_metrics.remove(target)
    in_progress.remove(target)

    return res

# Function to find all connected factors to a given target metric
def find_connected_factors(target, all_relationships, connected_factors):
    if target in connected_factors:
        return
    connected_factors.add(target)
    
    # Search upstream (where target is an input)
    inputs = all_relationships.get(target, {}).get('inputs', [])
    for inp in inputs:
        find_connected_factors(inp['name'], all_relationships, connected_factors)
    
    # Search downstream (where target is an output)
    for key, rel in all_relationships.items():
        outputs = rel.get('outputs', [])
        for output in outputs:
            if output['name'] == target:
                find_connected_factors(key, all_relationships, connected_factors)

import random

def debug_situations(all_relationships, situations, policy_values, rerun_number=2, initial_metrics_values=None):
    metrics_values = initial_metrics_values if initial_metrics_values is not None else {}
    metrics_values.update({"_globaleconomy_": 0.5, "_default_": 0.5, "_year": 0.5})

    reset_metrics = None

    for i in range(rerun_number):
        situation_score = 0
        good_situations = []
        bad_situations = []
        for situation, attributes in situations.items():
            situation_value = compute_metric(situation, all_relationships, policy_values, metrics_values, reset_metrics=reset_metrics)
            if situation_value >= all_relationships[situation]['start_trigger']:
                situation_score += 1 if attributes['positive'] else -1
                (good_situations if attributes['positive'] else bad_situations).append((situation, situation_value))
                        
        reset_metrics = set(metrics_values.keys())

    print("Good Situations (" + str(len(good_situations)) + "):", good_situations)
    print("Bad Situations (" + str(len(bad_situations)) + "):", bad_situations)
    print("Difference Score:", situation_score)
    return metrics_values

DEBUG_VALUE = [""]
policy_values = {key: round(random.uniform(0, 1), 2) for key in all_relationships if all_relationships[key].get('type') == 'policy'}
metrics_values = debug_situations(all_relationships, situations, policy_values)

Good Situations (1): [('EgalitarianSociety', 1)]
Bad Situations (10): [('TechnologyBackwater', 0.61), ('TeacherShortage', 1), ('BrainDrain', 1), ('TaxEvasion', 0.93), ('DroneWars', 1), ('LudditeRiots', 0.99), ('ForeignIntelligenceInterference', 1), ('Polarization', 1.0), ('FakeNews', 1), ('MediaMonopolies', 0.64)]
Difference Score: -9


In [6]:
all_relationships["EUContribution"]

{'type': 'situation',
 'inputs': [{'type': 'metric',
   'name': 'ForeignRelations',
   'equation': '0-(0.1*x)'}],
 'outputs': [{'type': 'metric',
   'name': 'Patriot_freq',
   'equation': '-0.05-(0*x)'},
  {'type': 'metric', 'name': 'Patriot', 'equation': '-0.05-(0*x)'},
  {'type': 'metric', 'name': 'InternationalTrade', 'equation': '0.1+(0*x)'},
  {'type': 'metric', 'name': 'Stability', 'equation': '0+(0.1*x)'},
  {'type': 'metric', 'name': 'ForeignInvestment', 'equation': '0+(0.1*x)'}],
 'start_trigger': 0.011,
 'stop_trigger': -1.0}

In [17]:
from pyswarm import pso

def optimize_situations(all_relationships, situations):
    last_10_scores = []
    last_100_scores = []

    def objective_function_pso(x, *args):
        nonlocal last_10_scores, last_100_scores
        policy_values = {key: round(x[i], 2) for i, key in enumerate(policy_factors)}
        metrics_values = {"_globaleconomy_": 0.5, "_default_": 0.5, "_year": 0.5}
        situation_score = 0
        reset_metrics = None

        for i in range(2):
            situation_score = 0
            for situation, attributes in situations.items():
                situation_value = compute_metric(situation, all_relationships, policy_values, metrics_values, reset_metrics=reset_metrics)
                if situation_value >= all_relationships[situation]['start_trigger']:
                    if attributes['positive']:
                        situation_score -= 1  # Minimize the negative to maximize positives
                            
            reset_metrics = set(metrics_values.keys())

        last_10_scores.append(situation_score)
        last_100_scores.append(situation_score)
        
        if len(last_10_scores) > 10:
            last_10_scores.pop(0)
            
        if len(last_100_scores) > 100:
            last_100_scores.pop(0)
        
        avg_10 = round(sum(last_10_scores) / len(last_10_scores), 2)    
        avg_100 = round(sum(last_100_scores) / len(last_100_scores), 2)
        
        print(f"Current Score: {situation_score}, Avg last 10: {avg_10}, Avg last 100: {avg_100}")        
        return situation_score 
    
    positive_situations = [name for name, data in situations.items() if data['positive']]
    negative_situations = [name for name, data in situations.items() if not data['positive']]

    all_connected_factors = set()
    for situation in positive_situations + negative_situations:
        find_connected_factors(situation, all_relationships, all_connected_factors)

    policy_factors = [factor for factor in all_connected_factors if all_relationships[factor].get('type') == 'policy']
    
    lb = [0 for _ in policy_factors]
    ub = [1 for _ in policy_factors]

    xopt, fopt = pso(objective_function_pso, lb, ub, args=(), swarmsize=100, omega=0.5, phip=0.5, phig=1.0, maxiter=10000, minstep=0.01, minfunc=0.01, debug=False)

    optimized_values = {key: xopt[i] for i, key in enumerate(policy_factors)}
    changed_metrics = []
    untouched_metrics = []
    good_situations = []
    bad_situations = []

    for key, value in optimized_values.items():
        if abs(value - 0.5) > 1e-4:
            changed_metrics.append((key, value))
        else:
            untouched_metrics.append(key)
            
    reset_metrics = None
            
    for i in range(2):
        situation_score = 0
        for situation, attributes in situations.items():
            situation_value = compute_metric(situation, all_relationships, optimized_values, metrics_values, reset_metrics=reset_metrics)
            if situation_value >= all_relationships[situation]['start_trigger']:
                situation_score += 1 if attributes['positive'] else -1
                (good_situations if attributes['positive'] else bad_situations).append((situation, situation_value))
                
        reset_metrics = set(metrics_values.keys())

    final_score = -fopt

    return optimized_values, changed_metrics, untouched_metrics, final_score, good_situations, bad_situations
    

optimized_values, changed_metrics, untouched_metrics, final_score, good_situations, bad_situations = optimize_situations(
    all_relationships, situations)
print("Changed Metrics:", changed_metrics)
print("Good Situations:", good_situations)
print("Bad Situations:", bad_situations)
print("Final Score:", final_score)

Current Score: -1, Avg last 10: -1.0, Avg last 100: -1.0
Current Score: -1, Avg last 10: -1.0, Avg last 100: -1.0
Current Score: -1, Avg last 10: -1.0, Avg last 100: -1.0
Current Score: -2, Avg last 10: -1.25, Avg last 100: -1.25
Current Score: -1, Avg last 10: -1.2, Avg last 100: -1.2
Current Score: -3, Avg last 10: -1.5, Avg last 100: -1.5
Current Score: -1, Avg last 10: -1.43, Avg last 100: -1.43
Current Score: 0, Avg last 10: -1.25, Avg last 100: -1.25
Current Score: -2, Avg last 10: -1.33, Avg last 100: -1.33
Current Score: -2, Avg last 10: -1.4, Avg last 100: -1.4
Current Score: -2, Avg last 10: -1.5, Avg last 100: -1.45
Current Score: -2, Avg last 10: -1.6, Avg last 100: -1.5
Current Score: -3, Avg last 10: -1.8, Avg last 100: -1.62
Current Score: -1, Avg last 10: -1.7, Avg last 100: -1.57
Current Score: -1, Avg last 10: -1.7, Avg last 100: -1.53
Current Score: -1, Avg last 10: -1.5, Avg last 100: -1.5
Current Score: -1, Avg last 10: -1.5, Avg last 100: -1.47
Current Score: -2, 

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1134, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 311, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2062, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2098, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [19]:
policy_values

{'AbortionLaw': 0.68,
 'AdultEducationSubsidies': 0.62,
 'AgricultureSubsidies': 0.91,
 'AirlineTax': 0.09,
 'AlcoholAwarenessCampaign': 0.72,
 'AlcoholLaw': 0.29,
 'AlcoholTax': 0.65,
 'AntibioticsBan': 0.5,
 'AntiCorruptionAgency': 0.66,
 'ArmedPolice': 0.53,
 'ArtsSubsidies': 0.6,
 'AutomationTax': 0.77,
 'BanCoal': 0.53,
 'Bandivorce': 0.59,
 'Banforeignchurchservice': 0.23,
 'Banhomosexuality': 0.95,
 'Banlowmpgcars': 0.52,
 'Banprivateeducation': 0.28,
 'Banprivatehealthcare': 0.1,
 'BanSecondHomeOwnership': 0.93,
 'BanSundayShopping': 0.92,
 'BicycleSubsidies': 0.69,
 'BiofuelSubsidies': 0.62,
 'BodyCameras': 0.14,
 'BorderControls': 0.58,
 'BorderWall': 0.7,
 'BusinessStartupCampaign': 0.95,
 'BusLanes': 0.56,
 'BusSubsidies': 0.07,
 'CapCEOPayMultiplier': 0.16,
 'CapitalGainsTax': 0.65,
 'CarbonCaptureAndStorage': 0.95,
 'CarbonTax': 0.63,
 'CarEmmissionsLimits': 0.56,
 'CarPoolingCampaign': 0.08,
 'CarTax': 0.49,
 'CCTVCameras': 0.22,
 'CharityTaxRelief': 0.79,
 'ChildBenefit

In [24]:
import json

# Read pre-set policy values from JSON file
with open('results/pso/minimize_negative-2.json', 'r') as f:
    preset_policy_values = json.load(f)

# Initialize an empty dictionary for metrics values
initial_metrics_values = {"_globaleconomy_": 0.5, "_default_": 0.5, "_year": 0.5}

# Run the computations twice and print the results
metrics_values = debug_situations(all_relationships, situations, preset_policy_values, 3, initial_metrics_values)

print("Metrics Values:", metrics_values)


Good Situations (3): [('EgalitarianSociety', 1), ('EndangeredSpecies', 0.47), ('PrivateSpaceProgram', 0.67)]
Bad Situations (2): [('DroneWars', 1), ('ForeignIntelligenceInterference', 1)]
Difference Score: 1
Metrics Values: {'_globaleconomy_': 0.5, '_default_': 0.5, '_year': 0.5, 'Retired_perc': 0, '_inv_ElectricCarTransition': 0, 'Poor_perc': 0, 'PrivateHousing': 0.0, 'Capitalist_perc': 0, 'Emigration': 0.53, 'IndustrialAutomation': 0.63, 'Technology': 0.61, 'PrivateEnergy': 0.0, 'PrivateHealthcare': 0.6, 'PrivateRail': 0.0, 'PrivateSchools': 1.0, 'PrivateTelecoms': 0.0, 'PrivateWater': 0.0, '_global_interest_rates_': 0, '_effectivedebt_': 0, 'Inflation': 0.2, 'EUContribution': 0.0, 'EUSubsidies': 0.0, 'EUMonetaryPolicy': 0, 'DebtCrisis': 0.0, 'Patriot_perc': 0, 'ExtremeNationalism': 0, 'ForeignRelations': 0.34, 'GenerationalWealthGap': 0.36, 'PrivatePensions': 0.12, 'Ghettos': 0, 'VigilanteMobs': 0, 'Homelessness': 0, 'WorkingWeek': 0.68, 'FakeNews': 0.59, 'InternetSpeed': 0.37, 'Med

In [22]:
import json

# Read JSON
filename= "results/run-3-negative.json"
with open(filename, "r") as f:
    data = json.load(f)

# Sort keys
sorted_data = {k: data[k] for k in sorted(data.keys())}

# Write JSON
with open(filename, "w") as f:
    json.dump(sorted_data, f, indent=4)


In [9]:
from collections import defaultdict
from itertools import combinations
import numpy as np
import os
import json

# Load all JSON data
all_data = []
for filename in os.listdir("results/pso"):
    if filename.endswith(".json"):
        with open(f"results/pso/{filename}", "r") as f:
            data = json.load(f)
            all_data.append(data)

# Initialize dictionaries
all_agree = {}
some_agree = {}
disagree = {}

# Check each policy key
for key in all_data[0].keys():
    values = np.array([d[key] for d in all_data])
    n_files = len(values)

    agreement_count = defaultdict(int)

    # Compare all combinations of pairs of files
    for i, j in combinations(range(n_files), 2):
        if abs(values[i] - values[j]) <= 0.1:
            agreement_count[i] += 1
            agreement_count[j] += 1

    # Classify based on the number of agreements
    if len(agreement_count) == n_files and all(count == n_files - 1 for count in agreement_count.values()):
        all_agree[key] = values.tolist()
    elif len(agreement_count) >= 2:
        some_agree[key] = values.tolist()
    else:
        disagree[key] = values.tolist()


In [10]:
all_agree

{'ArmedPolice': [0.63, 0.625, 0.62],
 'AutomationTax': [0.11, 0.09, 0.07],
 'BanCoal': [0.07, 0.065, 0.06],
 'Bandivorce': [0.0, 0.0, 0.0],
 'CCTVCameras': [1.0, 0.995, 0.99],
 'CapitalGainsTax': [0.0, 0.0, 0.0],
 'CarEmmissionsLimits': [0.0, 0.0, 0.0],
 'CharityTaxRelief': [0.88, 0.9, 0.92],
 'CitizenshipForSale': [0.0, 0.0, 0.0],
 'CleanEnergySubsidies': [0.77, 0.7250000000000001, 0.68],
 'CompulsorySchoolSports': [0.0, 0.0, 0.0],
 'Compulsoryworkfortheunemployed ': [0.24, 0.21, 0.18],
 'Cyberbullyingawarenesscampaign': [1.0, 0.98, 0.96],
 'DeathPenalty': [0.0, 0.03, 0.06],
 'Diversityquotasforcompanies': [0.6, 0.56, 0.52],
 'DivertedProfitsTax': [0.71, 0.675, 0.64],
 'DroneStrikeAct': [0.7, 0.655, 0.61],
 'EcoHomeRegulations': [0.09, 0.045, 0.0],
 'ForeignInvestmentRestrictions': [0.19, 0.14500000000000002, 0.1],
 'FreeBusPasses': [1.0, 1.0, 1.0],
 'FreeParentingClasses': [0.31, 0.33499999999999996, 0.36],
 'HealthcareVouchers': [0.9, 0.95, 1.0],
 'InheritanceTax': [0.2, 0.22, 0.24]

In [11]:
some_agree

{'AdultEducationSubsidies': [1.0, 0.97, 0.7],
 'AlcoholAwarenessCampaign': [1.0, 0.8, 0.84],
 'AlcoholTax': [0.24, 0.0, 0.0],
 'BanSundayShopping': [0.29, 0.31, 1.0],
 'Banhomosexuality': [0.71, 0.33, 0.43],
 'Banlowmpgcars': [0.43, 0.42, 0.71],
 'Banprivateeducation': [0.24, 0.0, 0.0],
 'Banprivatehealthcare': [0.69, 0.09, 0.0],
 'BicycleSubsidies': [0.16, 0.88, 0.82],
 'BodyCameras': [0.77, 0.55, 0.63],
 'BorderControls': [0.43, 0.15, 0.25],
 'BusSubsidies': [1.0, 0.95, 0.32],
 'CapCEOPayMultiplier': [0.14, 0.1, 0.56],
 'CarbonCaptureAndStorage': [0.0, 0.02, 0.92],
 'CarbonTax': [0.46, 1.0, 0.9],
 'ChildBenefit': [0.18, 1.0, 0.99],
 'ChildcareProvision': [0.24, 0.55, 0.65],
 'CityFarms': [1.0, 0.78, 0.84],
 'ClimateChangeAdaptionFund': [0.84, 0.8, 0.47],
 'Closeairportscompletely': [0.62, 0.52, 0.4],
 'CompulsoryForeignLanguageClasses': [0.45, 0.29, 0.32],
 'CorporationTax': [0.76, 0.0, 0.05],
 'Creationism': [0.15, 0.07, 1.0],
 'DetentionWithoutTrial': [0.38, 0.0, 0.0],
 'Diplomatic

In [12]:
disagree

{'AbortionLaw': [0.47, 0.78, 0.0],
 'AgricultureSubsidies': [0.43, 0.54, 0.19],
 'AirlineTax': [0.15, 0.44, 0.0],
 'AlcoholLaw': [0.0, 1.0, 0.74],
 'AntiCorruptionAgency': [0.56, 0.0, 1.0],
 'AntibioticsBan': [0.56, 0.74, 1.0],
 'ArtsSubsidies': [0.19, 0.72, 1.0],
 'BanSecondHomeOwnership': [0.63, 0.81, 0.27],
 'Banforeignchurchservice': [0.63, 0.5, 0.1],
 'BiofuelSubsidies': [0.67, 0.93, 0.29],
 'BorderWall': [0.39, 0.6, 0.0],
 'BusLanes': [0.86, 0.73, 0.4],
 'CarPoolingCampaign': [0.24, 0.08, 0.73],
 'CarTax': [0.55, 0.21, 0.35],
 'CitizenshipTests': [0.39, 0.0, 0.86],
 'CleanFuelSubsidy': [0.98, 0.75, 0.1],
 'CommunityPolicing': [0.6, 0.91, 0.0],
 'CompulsoryFoodLabelling': [0.37, 0.8, 0.23],
 'Compulsorychurchattendance': [0.04, 0.73, 0.99],
 'Compulsorylanguagelessons': [0.39, 0.99, 0.15],
 'CongestionCharging': [0.73, 0.0, 1.0],
 'ConsumerRights': [0.82, 0.61, 0.38],
 'Curfews': [0.62, 0.36, 1.0],
 'CyclingCampaign': [0.38, 1.0, 0.54],
 'DrugTreatment': [0.63, 0.0, 0.29],
 'Enter

In [17]:
len(disagree)

213

In [8]:
# Take the values they agree on and try to find an optimum for metrics where they don't
from pyswarm import pso
import numpy as np

def optimize_situations_with_fixed_policies(all_relationships, situations, all_agree, disagree):
    last_10_scores = []
    last_100_scores = []
    
    def objective_function_pso(x, *args):
        nonlocal last_10_scores, last_100_scores
        optimized_policy_values = {key: round(x[i], 2) for i, key in enumerate(tweakable_policy_factors)}
        complete_policy_values = {**optimized_policy_values, **fixed_policy_values}
        
        situation_score = 0
        reset_metrics = None

        for i in range(2):
            situation_score = 0
            for situation, attributes in situations.items():
                situation_value = compute_metric(situation, all_relationships, complete_policy_values, metrics_values, reset_metrics=reset_metrics)
                if situation_value >= all_relationships[situation]['start_trigger']:
                    if not attributes['positive']:
                        situation_score += 1  # Maximize the negative to minimize negatives
            
            reset_metrics = set(metrics_values.keys())
            
        last_10_scores.append(situation_score)
        last_100_scores.append(situation_score)
        
        if len(last_10_scores) > 10:
            last_10_scores.pop(0)
            
        if len(last_100_scores) > 100:
            last_100_scores.pop(0)
        
        avg_10 = round(sum(last_10_scores) / len(last_10_scores), 2)    
        avg_100 = round(sum(last_100_scores) / len(last_100_scores), 2)
        
        print(f"Current Score: {situation_score}, Avg last 10: {avg_10}, Avg last 100: {avg_100}")        

        
        return situation_score

    all_connected_factors = set()
    for situation in situations.keys():
        find_connected_factors(situation, all_relationships, all_connected_factors)

    tweakable_policy_factors = [factor for factor in all_connected_factors if all_relationships[factor].get('type') == 'policy' and factor in disagree.keys()]
    fixed_policy_factors = [factor for factor in all_connected_factors if factor in all_agree.keys()]

    fixed_policy_values = {key: np.mean(all_agree[key]) for key in fixed_policy_factors}

    lb = [0 for _ in tweakable_policy_factors]
    ub = [1 for _ in tweakable_policy_factors]
    metrics_values = {"_globaleconomy_": 0.5, "_default_": 0.5, "_year": 0.5}

    xopt, fopt = pso(objective_function_pso, lb, ub, args=(), swarmsize=100, omega=0.5, phip=0.5, phig=1.0, maxiter=10000, minstep=0.01, minfunc=0.01, debug=False)

    optimized_values = {key: round(xopt[i], 2) for i, key in enumerate(tweakable_policy_factors)}
    optimized_values.update(fixed_policy_values)

    bad_situations = []
    reset_metrics = None

    for i in range(2):
        situation_score = 0
        for situation, attributes in situations.items():
            situation_value = compute_metric(situation, all_relationships, optimized_values, metrics_values, reset_metrics=reset_metrics)
            if situation_value >= all_relationships[situation]['start_trigger']:
                if not attributes['positive']:
                    bad_situations.append((situation, situation_value))

        reset_metrics = set(metrics_values.keys())

    final_score = fopt

    return optimized_values, final_score, bad_situations

optimized_values, final_score, bad_situations = optimize_situations_with_fixed_policies(all_relationships, situations, all_agree, disagree)
print("Optimized Values:", optimized_values)
print("Bad Situations:", bad_situations)
print("Final Score:", final_score)


Current Score: 8, Avg last 10: 8.0, Avg last 100: 8.0
Current Score: 13, Avg last 10: 10.5, Avg last 100: 10.5
Current Score: 11, Avg last 10: 10.67, Avg last 100: 10.67
Current Score: 11, Avg last 10: 10.75, Avg last 100: 10.75
Current Score: 12, Avg last 10: 11.0, Avg last 100: 11.0
Current Score: 14, Avg last 10: 11.5, Avg last 100: 11.5
Current Score: 11, Avg last 10: 11.43, Avg last 100: 11.43
Current Score: 11, Avg last 10: 11.38, Avg last 100: 11.38
Current Score: 10, Avg last 10: 11.22, Avg last 100: 11.22
Current Score: 10, Avg last 10: 11.1, Avg last 100: 11.1
Current Score: 10, Avg last 10: 11.3, Avg last 100: 11.0
Current Score: 13, Avg last 10: 11.3, Avg last 100: 11.17
Current Score: 12, Avg last 10: 11.4, Avg last 100: 11.23
Current Score: 10, Avg last 10: 11.3, Avg last 100: 11.14
Current Score: 14, Avg last 10: 11.5, Avg last 100: 11.33
Current Score: 12, Avg last 10: 11.3, Avg last 100: 11.38
Current Score: 12, Avg last 10: 11.4, Avg last 100: 11.41
Current Score: 10,

In [15]:
len(some_agree)

99

In [16]:
import json
import numpy as np

# Compute the average for each policy in all_agree array and round to 2 decimals
avg_all_agree = {key: round(np.mean(values), 2) for key, values in all_agree.items()}

# Load current French policies
with open("results/current_french_policies.json", 'r') as f:
    current_policies = json.load(f)

# Compare current policies with average "all_agree" values and print if difference is > 0.1
for key, current_value in current_policies.items():
    if key in avg_all_agree:
        avg_value = avg_all_agree[key]
        if abs(current_value - avg_value) > 0.1:
            print(f"{key}: Current Value = {current_value}, Suggested Value = {avg_value}")


CapitalGainsTax: Current Value = 0.18, Suggested Value = 0.0
CarEmmissionsLimits: Current Value = 0.5, Suggested Value = 0.0
CCTVCameras: Current Value = 0.3, Suggested Value = 1.0
CharityTaxRelief: Current Value = 0.34, Suggested Value = 0.9
CleanEnergySubsidies: Current Value = 0.1, Suggested Value = 0.73
DivertedProfitsTax: Current Value = 0.52, Suggested Value = 0.68
InheritanceTax: Current Value = 0.4, Suggested Value = 0.22
JudiciaryIndependence: Current Value = 0.85, Suggested Value = 0.48
MaternityLeave: Current Value = 0.3, Suggested Value = 0.0
MilitarySpending: Current Value = 0.55, Suggested Value = 0.97
Prisons: Current Value = 0.4, Suggested Value = 0.86
SelectiveSchooling: Current Value = 0.5, Suggested Value = 0.0
StateHousing: Current Value = 0.45, Suggested Value = 0.55
TollRoads: Current Value = 0.66, Suggested Value = 0.97
WorkersOnBoards: Current Value = 0.33, Suggested Value = 0.0


In [18]:
# Load minimize_negative-2.json values
with open("results/pso/minimize_negative-2.json", 'r') as f:
    minimize_negative = json.load(f)

# Compare current policies with minimize_negative values and print if difference is > 0.1
for key, current_value in current_policies.items():
    if key in minimize_negative:
        min_value = minimize_negative[key]
        if abs(current_value - min_value) > 0.1:
            print(f"{key}: Current Value = {current_value}, Suggested Value = {min_value}")


AdultEducationSubsidies: Current Value = 0.6, Suggested Value = 0.97
AgricultureSubsidies: Current Value = 0.7, Suggested Value = 0.54
AirlineTax: Current Value = 0.02, Suggested Value = 0.44
AlcoholLaw: Current Value = 0.5, Suggested Value = 1.0
AntibioticsBan: Current Value = 0.6, Suggested Value = 0.74
AntiCorruptionAgency: Current Value = 0.5, Suggested Value = 0.0
BanSundayShopping: Current Value = 0.1, Suggested Value = 0.31
BicycleSubsidies: Current Value = 0.1, Suggested Value = 0.88
BiofuelSubsidies: Current Value = 0.5, Suggested Value = 0.93
BorderControls: Current Value = 0.5, Suggested Value = 0.15
BusLanes: Current Value = 0.6, Suggested Value = 0.73
BusSubsidies: Current Value = 0.25, Suggested Value = 0.95
CapitalGainsTax: Current Value = 0.18, Suggested Value = 0.0
CarbonTax: Current Value = 0.29, Suggested Value = 1.0
CarEmmissionsLimits: Current Value = 0.5, Suggested Value = 0.0
CCTVCameras: Current Value = 0.3, Suggested Value = 0.995
CharityTaxRelief: Current Valu

In [19]:
# Create a list of tuples containing the policy key, current value, and suggested value
discrepancies = []
for key, current_value in current_policies.items():
    if key in minimize_negative:
        min_value = minimize_negative[key]
        if abs(current_value - min_value) > 0.1:
            discrepancies.append((key, current_value, min_value))

# Sort the list by the absolute distance of suggested value from 0.5, in descending order
discrepancies.sort(key=lambda x: abs(x[2] - 0.5), reverse=True)

# Print the sorted list
for key, current_value, suggested_value in discrepancies:
    print(f"{key}: Current Value = {current_value}, Suggested Value = {suggested_value}")


AlcoholLaw: Current Value = 0.5, Suggested Value = 1.0
AntiCorruptionAgency: Current Value = 0.5, Suggested Value = 0.0
CapitalGainsTax: Current Value = 0.18, Suggested Value = 0.0
CarbonTax: Current Value = 0.29, Suggested Value = 1.0
CarEmmissionsLimits: Current Value = 0.5, Suggested Value = 0.0
ChildBenefit: Current Value = 0.4, Suggested Value = 1.0
CitizenshipTests: Current Value = 0.25, Suggested Value = 0.0
CorporationTax: Current Value = 0.62, Suggested Value = 0.0
DrugTreatment: Current Value = 0.75, Suggested Value = 0.0
FamilyPlanning: Current Value = 0.5, Suggested Value = 1.0
FoodStandards: Current Value = 0.5, Suggested Value = 0.0
ForeignAid: Current Value = 0.45, Suggested Value = 1.0
Gambling: Current Value = 0.1, Suggested Value = 1.0
GayMarriage: Current Value = 0.5, Suggested Value = 1.0
JunkFoodTax: Current Value = 0.5, Suggested Value = 1.0
MaternityLeave: Current Value = 0.3, Suggested Value = 0.0
NuclearWeapons: Current Value = 0.66, Suggested Value = 1.0
Payro

In [23]:
# Create a list of tuples containing the policy key and suggested value for keys only in all_agree
exclusive_keys = []
for key, avg_value in minimize_negative.items():
    if key not in current_policies and key not in avg_all_agree:
        exclusive_keys.append((key, avg_value))

# Sort the list by the suggested value, in descending order
exclusive_keys.sort(key=lambda x: x[1], reverse=True)

# Print the sorted list
for key, suggested_value in exclusive_keys:
    print(f"{key}: Suggested Value = {suggested_value}")


DriverlessCarLaws: Suggested Value = 1.0
bansocialmedia: Suggested Value = 1.0
SocialJusticeFoundation: Suggested Value = 1.0
SolidarityTax: Suggested Value = 1.0
banwomendriving: Suggested Value = 1.0
TearGas: Suggested Value = 1.0
CyclingCampaign: Suggested Value = 1.0
Tradeunionbansatcompanies: Suggested Value = 1.0
packagingtax: Suggested Value = 1.0
SyntheticMeatResearchGrants: Suggested Value = 1.0
Compulsorylanguagelessons: Suggested Value = 0.99
RareEarthRefinement: Suggested Value = 0.99
watermeters: Suggested Value = 0.98
FreeEyeTests: Suggested Value = 0.98
InternetCensorship: Suggested Value = 0.97
FinancialTransactionsTax: Suggested Value = 0.97
bancrypto: Suggested Value = 0.96
competitionlaw: Suggested Value = 0.96
DiplomaticService: Suggested Value = 0.92
YouthPoliticsCouncil: Suggested Value = 0.87
Publicreligiousbroadcasts: Suggested Value = 0.86
Publictaxreturns: Suggested Value = 0.83
MarsProgram: Suggested Value = 0.82
YoungEntrepreneurScheme: Suggested Value = 0.8